In [1]:
# Import TensorFlow library for deep learning and neural network operations
import tensorflow


In [2]:
# Import necessary Keras layers for building the CNN model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization  

# Import Sequential model type from Keras
from keras.models import Sequential  

# Import utility for generating augmented image datasets
from keras.preprocessing.image import ImageDataGenerator  

# Import callbacks for early stopping and saving the best model during training
from keras.callbacks import EarlyStopping, ModelCheckpoint  


In [3]:
# Build a Convolutional Neural Network (CNN) using Sequential API

cnn = Sequential([
    
    # First convolutional block
    Conv2D(64, (3,3), activation='relu', input_shape=(64,64,3)),
    MaxPooling2D(pool_size=(2,2)),
    
    # Second convolutional block
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    # Second convolutional block
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    # Fully connected layers
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(0.5),
    
    Dense(64, activation='relu'),
    Dropout(0.5),
    
    Dense(32, activation='relu'),
    Dropout(0.5),

    # Output layer for binary classification
    Dense(1, activation='sigmoid')
])




In [4]:
# Import Adam optimizer from Keras
from keras.optimizers import Adam  

# Initialize Adam optimizer with a low learning rate for stable training
Adam = Adam(learning_rate=0.0001)  

# Compile the CNN model with binary crossentropy loss and accuracy metric
cnn.compile(loss="binary_crossentropy", optimizer=Adam, metrics=['accuracy'])  


In [ ]:
# Create image data generator for training with augmentation
train_datagen = ImageDataGenerator(
        rescale=1./255,        # Normalize pixel values to [0,1]
        shear_range=0.2,       # Apply random shearing
        zoom_range=0.2,        # Apply random zoom
        horizontal_flip=True   # Randomly flip images horizontally
)

# Create image data generator for testing (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess training images from directory
train_generator = train_datagen.flow_from_directory(
        "/kaggle/input/cat-and-dog/training_set/training_set",
        target_size=(64, 64),   # Resize images to match model input
        batch_size=32,
        class_mode='binary'    # Binary classification labels
)

# Load and preprocess testing images from directory
test_generator = test_datagen.flow_from_directory(
        "/kaggle/input/cat-and-dog/test_set/test_set",
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary'
)

# Callback to save the best model based on validation accuracy
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')

# Callback to stop training early if validation loss doesn't improve
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the CNN model with data generators and callbacks
cnn.fit_generator(
        train_generator,
        steps_per_epoch=250,    # Number of batches per training epoch
        validation_steps=63,    # Number of batches for validation
        epochs=50,
        validation_data=test_generator,
        callbacks=[checkpoint, early_stop]
)

# Save the final trained model
cnn.save('final_model.h5')


Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


C:\Users\Aamir Shahzad\AppData\Local\Temp\ipykernel_10388\4140155047.py:35: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  cnn.fit_generator(


Epoch 1/50
250/250 [==============================] - 362s 1s/step - loss: 0.6941 - accuracy: 0.4987 - val_loss: 0.6932 - val_accuracy: 0.4975
Epoch 2/50
250/250 [==============================] - 309s 1s/step - loss: 0.6931 - accuracy: 0.5087 - val_loss: 0.6932 - val_accuracy: 0.4990
Epoch 3/50
250/250 [==============================] - 308s 1s/step - loss: 0.6932 - accuracy: 0.5085 - val_loss: 0.6932 - val_accuracy: 0.5010
Epoch 4/50
250/250 [==============================] - 307s 1s/step - loss: 0.6933 - accuracy: 0.4967 - val_loss: 0.6931 - val_accuracy: 0.5159
Epoch 5/50
250/250 [==============================] - 311s 1s/step - loss: 0.6920 - accuracy: 0.5131 - val_loss: 0.6909 - val_accuracy: 0.4995
Epoch 6/50
250/250 [==============================] - 308s 1s/step - loss: 0.6905 - accuracy: 0.5268 - val_loss: 0.6869 - val_accuracy: 0.5283
Epoch 7/50
250/250 [==============================] - 310s 1s/step - loss: 0.6878 - accuracy: 0.5554 - val_loss: 0.6824 - val_accuracy: 0.5060

In [ ]:
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

# Load and preprocess the image
img = load_img("test_set\test_set\dogs\dog.4123.jpg", 
               target_size=(64, 64))
img = img_to_array(img)         # Convert to array
img = img / 255.0               # Normalize pixel values
img = np.expand_dims(img, axis=0)  # Add batch dimension

# Make prediction
prediction = cnn.predict(img)
prob = prediction[0][0]  # Probability of being dog

# Assign class based on probability
if prob > 0.5:
    predicted_class = 'dog'
else:
    predicted_class = 'cat'

# Display results
print(f"Predicted class: {predicted_class}")
print(f"Probability of being dog: {prob:.4f}")


1/1 [==============================] - 0s 220ms/step
Predicted class: dog
Probability of being dog: 0.9644
